# LangChain Quickstart
- Code 출처 : https://python.langchain.com/docs/get_started/quickstart 
- 수정사항 : 설명과 프롬프트 내용을 한글로 변경 

### LangChain 설치

In [1]:
# langchain-community 0.0.19 설치 이유 - ModuleNotFoundError: No module named 'pwd' 해결 
%pip install langchain langchain-openai langchain-community==0.0.19

Note: you may need to restart the kernel to use updated packages.


## 1. LLM Chain

In [2]:
# import os
# os.environ["OPENAI_API_KEY"] = "<your OpenAI API key if not set as env var>"

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [4]:
llm.invoke("langsmith가 어떻게 테스트에 도움을 줄 수 있나요?")

AIMessage(content='Langsmith는 다양한 언어 및 문법 검사 기능을 제공하여 테스트 중에 발생할 수 있는 언어 관련 문제를 식별하고 수정하는 데 도움을 줄 수 있습니다. 또한 Langsmith는 텍스트 데이터를 분석하고 해석하여 테스트 케이스를 작성하거나 결과를 분석하는 데 도움을 줄 수 있습니다. 또한 Langsmith는 다양한 자연어 처리 기능을 제공하여 테스트 케이스를 자동화하거나 자연어 처리 시스템을 테스트하는 데 도움을 줄 수 있습니다. 따라서 Langsmith를 사용하면 테스트의 효율성과 정확성을 높일 수 있습니다.')

#### Prompt template : 사용자 입력을 LLM에 더 나은 입력으로 변환하는 데 사용

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 세계적인 수준의 기술 문서 작성자입니다."),
    ("user", "{input}")
])

#### llm과 prompt를 간단한 LLM chain으로 결합

In [6]:
chain = prompt | llm 

In [7]:
chain.invoke({"input": "langsmith가 어떻게 테스트에 도움을 줄 수 있나요?"})

AIMessage(content='langsmith는 다양한 방식으로 테스트에 도움을 줄 수 있습니다. \n\n1. 자동화된 언어 테스트: langsmith는 다국어 환경에서 소프트웨어 테스트를 자동화할 수 있는 기술을 제공합니다. 이를 통해 다국어 지원 소프트웨어의 테스트를 보다 효율적으로 수행할 수 있습니다.\n\n2. 다국어 테스트 스트래티지 수립: langsmith는 다국어 소프트웨어의 테스트를 위한 전략 수립에 도움을 줄 수 있습니다. 언어 및 문화에 따른 차이를 고려한 테스트 계획 및 실행 방법을 제안하여 소프트웨어의 다국어 지원을 보다 효과적으로 검증할 수 있습니다.\n\n3. 다국어 테스트 케이스 개발: langsmith는 다국어 소프트웨어를 위한 테스트 케이스를 개발하는 데 도움을 줄 수 있습니다. 언어 및 지역에 따른 다양한 테스트 케이스를 고려하여 소프트웨어의 다국어 지원을 체계적으로 검증할 수 있습니다.\n\n4. 다국어 테스트 리소스 제공: langsmith는 다양한 언어 및 지역에 대한 테스트 리소스를 제공할 수 있습니다. 번역된 텍스트, 로컬라이즈된 콘텐츠 등을 활용하여 소프트웨어의 다국어 지원을 효과적으로 테스트할 수 있습니다.\n\n이러한 방법을 통해 langsmith는 다국어 소프트웨어의 품질을 향상시키고 사용자들에게 더 나은 다국어 환경을 제공할 수 있습니다.')

 #### 채팅 Message를 문자열로 변환하는 간단한 OutputParser를 추가

In [8]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [9]:
chain = prompt | llm | output_parser
chain.invoke({"input": "langsmith가 어떻게 테스트에 도움을 줄 수 있나요?"})

'langsmith는 테스트 작업에서 다양한 방법으로 도움을 줄 수 있습니다. 예를 들어:\n\n1. 테스트 케이스 작성: langsmith는 자연어 처리 기술을 사용하여 테스트 케이스를 작성하고 자동화할 수 있습니다. 이를 통해 효율적으로 테스트를 수행할 수 있습니다.\n\n2. 테스트 데이터 생성: langsmith는 특정한 테스트 시나리오에 맞는 테스트 데이터를 생성할 수 있습니다. 이를 통해 다양한 시나리오에서의 테스트를 보다 체계적으로 수행할 수 있습니다.\n\n3. 오류 분석: langsmith는 오류 메시지나 로그를 분석하여 오류의 원인을 식별하고 해결할 수 있는데 도움을 줄 수 있습니다.\n\n4. 테스트 리포팅: langsmith는 테스트 실행 결과를 분석하여 테스트 리포트를 작성할 수 있습니다. 이를 통해 테스트 결과를 시각화하고 보고서로 작성할 수 있습니다.\n\n이러한 방법을 통해 langsmith는 효율적이고 효과적인 테스트 수행을 지원할 수 있습니다.'

## 2. Retrieval Chain
- 질문에 제대로 답하려면 추가 컨텍스트를 제공해야 합니다. 검색을 통해 이를 수행할 수 있습니다. 
- Retrieval(검색)은 LLM에 직접 전달하기에는 너무 많은 데이터가 있을 때 유용합니다. 
- 관련 데이터에서 Retriever를 사용하여 가장 관련성이 높은 부분만 가져와서 프롬프트에 전달합니다. 
- 아래 코드는 벡터스토어에 데이터를 저장한 후, 이를 검색하는 예제입니다.

#### BeautifulSoup을 설치하고, WebBaseLoader로 웹페이지 데이터 로드

In [10]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

### 데이터를 임베딩벡터로 변환하여서 벡터스토어로 색인화

In [15]:
# FAISS(Fast Approximate Nearest Neighbor Search) 설치
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [16]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [17]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

### Retrieval Chain 생성

In [18]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""제공된 context에만 근거하여 다음 질문에 답하세요.:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [19]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "LangSmith가 어떻게 테스트에 도움을 줄 수 있나요?",
    "context": [Document(page_content="LangSmith를 사용하면 테스트 결과를 시각화할 수 있습니다.")]
})

'LangSmith를 사용하면 테스트 결과를 시각화할 수 있어서 테스트의 진행 상황을 더 쉽게 파악하고 분석할 수 있습니다. 결과적으로 테스트의 품질을 높이고 효율적으로 테스트를 수행할 수 있게 도와줄 수 있습니다.'

In [20]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [22]:
response = retrieval_chain.invoke({"input": "LangSmith가 어떻게 테스트에 도움을 줄 수 있나요?"})
print(response["answer"])

LangSmith는 디버깅, 테스트, 평가, 모니터링을 지원하여 LLM 애플리케이션을 구축하는 데 도움을 줄 수 있습니다. LangSmith는 모든 LLM 프레임워크에서 구축된 체인 및 지능형 에이전트를 테스트하고 평가할 수 있도록 허용하며, LangChain과 완벽하게 통합되어 있어 LLM을 사용하여 개발할 때 유용하게 사용할 수 있습니다.


## 3. Conversation Retrieval Chain
- LLM 주요 애플리케이션인 채팅봇 구현시에는 대화 히스토리를 고려해야 합니다.
- create_retrieval_chain 함수 사용시 2가지를 변경해야 합니다:
    - 검색 방법은 이제 가장 최근의 입력만 처리하는 것이 아니라 전체 히스토리를 고려해야 합니다.  
    - 최종 LLM 체인도 마찬가지로 전체 히스토리를 고려해야 합니다.

#### Updating Retrieval  
- 검색을 업데이트하기 위해 새 체인을 생성합니다.  
- 이 체인은 가장 최근의 입력(input)과 대화 기록(chat_history)을 받아 LLM을 사용하여 검색 쿼리를 생성합니다.

In [23]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation",
        ),
    ]
)

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

#### 사용자가 후속 질문을 하는 인스턴스를 전달하여 이를 테스트해 볼 수 있습니다.

In [24]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="LangSmith가 LLM 애플리케이션 테스트를 도와줄 수 있나요?"),
    AIMessage(content="예!"),
]

retriever_chain.invoke({"chat_history": chat_history, "input": "방법을 알려주세요."})

[Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Setup: Learn 

- 이렇게 하면 LangSmith에서 테스트에 대한 문서가 반환되는 것을 볼 수 있습니다. 
- 이는 LLM이 채팅 히스토리와 후속 질문을 결합하여 새로운 쿼리를 생성했기 때문입니다.
- 이제 새로운 retriever가 생겼으니 검색된 문서를 염두에 두고 대화를 계속할 수 있는 새로운 체인을 만들 수 있습니다.

In [25]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
    ]
)

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

#### End-to-end로 테스트할 수 있습니다.

In [26]:
chat_history = [
    HumanMessage(content="LangSmith가 LLM 애플리케이션 테스트를 도와줄 수 있나요?"),
    AIMessage(content="예!"),
]

retrieval_chain.invoke({"chat_history": chat_history, "input": "방법을 알려주세요."})

{'chat_history': [HumanMessage(content='LangSmith가 LLM 애플리케이션 테스트를 도와줄 수 있나요?'),
  AIMessage(content='예!')],
 'input': '방법을 알려주세요.',
 'context': [Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to

## 4. Agent
- Agent는 LLM이 수행할 단계를 결정합니다.
- Agent를 만들 때 Agent가 액세스할 수 있는 Tool을 정합니다.
- 이 예에서는 에이전트에게 2개의 Tool에 대한 액세스 권한을 부여하겠습니다:
    1. 방금 만든 Retriever : LangSmith에 대한 질문에 답변
    2. Search tool : 최신 정보가 필요한 질문에 답변  

#### Retriever를 위한 툴을 설정

In [27]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

#### Search를 위한 툴 설정
- Search 엔진으로 Tavily 사옹 : API Key - https://app.tavily.com/
- export TAVILY_API_KEY=...

In [29]:
# import os

# os.environ["TAVILY_API_KEY"] = "<your Tavily API key if not set as env var>"

In [32]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

#### Tool 리스트

In [33]:
tools = [retriever_tool, search]

 #### 미리 정의된 프롬프트를 사용하기 위해 langchain hub 설치

In [36]:
%pip install langchainhub

Note: you may need to restart the kernel to use updated packages.


In [38]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

#### Agent를 호출하고 Agent의 응답을 확인합니다.

In [39]:
agent_executor.invoke({"input": "LangSmith가 테스트에 어떤 도움을 줄 수 있나요?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'How can LangSmith help with tests?'}`


LangSmith | 🦜️🛠️ LangSmith





Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start​Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps​Check out the following sections to learn more about LangSmith:User Guide: Learn about the workflo

{'input': 'LangSmith가 테스트에 어떤 도움을 줄 수 있나요?',
 'output': 'LangSmith는 테스트에 도움을 주는 플랫폼입니다. LangSmith를 사용하면 LLM(언어 모델) 프레임워크를 기반으로 구축된 체인 및 지능형 에이전트를 디버깅, 테스트, 평가 및 모니터링할 수 있습니다. LangSmith는 LangChain과 원활하게 통합되며, LangChain은 LLM과 관련된 오픈 소스 프레임워크입니다. LangSmith를 사용하여 테스트를 수행하고 결과를 평가할 수 있습니다. 자세한 내용은 LangSmith의 사용자 가이드와 관련 섹션을 확인하시기 바랍니다.'}

In [40]:
agent_executor.invoke({"input": "오늘 서울 날씨는?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '오늘 서울 날씨'}`


[{'url': 'https://www.accuweather.com/ko/kr/seoul/226081/hourly-weather-forecast/226081', 'content': '오늘 서울의 날씨는 33°C에서 28°C의 약간 흐림 바람 동남동에서 30°C에서 22°C의 약간 흐림 바람 남남동으로 변化하는 것을 보여줍니다. 오후 9시부터 11시까지는 31°C에서 25°C의 약간 흐림 바람 동남동에서 28°C에서 22°C의 약간 흐림 바람 남남동으로 변化하는 것을 예'}, {'url': 'https://www.weather.go.kr/w/index.do', 'content': '기상청 날씨누리. 전국특보 발효중. 밀물·썰물 정보 해와 달이 뜨고지는 시간 기후정보포털 기상자료 개방포털 기상기후 빅데이터 대기질 예·경보 날씨해설 (유튜브) 기상청 행정누리집 산악날씨 한강홍수통제소 해양기상 정보포털 항공기상청. 아이콘 ...'}, {'url': 'https://www.weather.go.kr/w/weather/forecast/mid-term.do?stnId1=109', 'content': '충청남·북도 전라남·북도 경상남·북도 제주특별자치도 기상전망 발표시각 2024년 02월 16일 (금)요일 18:00 발표 예보관 (서정국) 서울ㆍ인천ㆍ경기도 (강수) 21일 (수)은 비 또는 눈이 내리겠습니다. (기온) 아침 기온은 20일 (화)~22일 (목)에 -2~5도로 평년 (최저기온 -6~-2도)보다 높겠고, 23일 (금)~26일 (월)에 -5~0도로 평년 (최저기온 -5~0도)과 비슷하겠습니다. 낮 기온은 20일 (화)에 9~10도로 평년 (최고기온 6~7도)보다 높겠고, 21일 (수)~26일 (월)에 4~8도로 평년 (최고기온 6~9도)보다 조금 낮겠습니다.'}, {'ur

{'input': '오늘 서울 날씨는?',
 'output': '서울의 오늘 날씨는 약간 흐림이며, 최고 기온은 33°C이고 최저 기온은 28°C입니다. 동남동에서 불어오는 바람이 30°C에서 22°C의 약간 흐림으로 변화할 것으로 예상됩니다. 오후 9시부터 11시까지는 31°C에서 25°C의 약간 흐림으로 변화할 것으로 예상됩니다.더 자세한 정보는 [링크](https://www.accuweather.com/ko/kr/seoul/226081/hourly-weather-forecast/226081)에서 확인하실 수 있습니다.'}

In [41]:
chat_history = [
    HumanMessage(content="LangSmith가 LLM 애플리케이션 테스트를 도와줄 수 있나요?"),
    AIMessage(content="예!"),
]
agent_executor.invoke({"chat_history": chat_history, "input": "방법을 알려주세요."})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LLM 애플리케이션 테스트 도와줄 방법'}`


LangSmith | 🦜️🛠️ LangSmith





Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start​Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps​Check out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith 

{'chat_history': [HumanMessage(content='LangSmith가 LLM 애플리케이션 테스트를 도와줄 수 있나요?'),
  AIMessage(content='예!')],
 'input': '방법을 알려주세요.',
 'output': 'LangSmith는 LLM 애플리케이션의 디버깅, 테스트, 평가 및 모니터링을 지원하는 플랫폼입니다. LangSmith는 LangChain과 원활하게 통합되어 있으며, LLM 프레임워크를 사용하여 구축된 체인 및 지능형 에이전트를 지원합니다. LangSmith를 사용하면 LLM 애플리케이션 라이프사이클 각 단계에서 지원하는 워크플로우, 계정 생성, API 키 획득, 환경 구성 등에 대해 자세히 알아볼 수 있습니다. 또한 LangSmith는 트레이싱 및 평가 기능을 제공합니다. 자세한 내용은 LangSmith의 문서를 참고하시기 바랍니다.'}

## 5. Serving with LangServe
- langserve, fastapi를 설치하고 로컬에서 실행 : python server.py

In [47]:
%pip install langserve fastapi sse_starlette

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\mask_rcnn-2.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


#### 아래 코드는 실행하지 마세요.  
#### server.py 파일의 내용입니다.

In [ ]:
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage
from langserve import add_routes


# 1. Load Retriever
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()


# 2. Create Tools
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
search = TavilySearchResults()
tools = [retriever_tool, search]


# 3. Create Agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


# 4. App definition
app = FastAPI(
    title="LangChain Server",
    version="1.0",
    description="A simple API server using LangChain's Runnable interfaces",
)


# 5. Adding chain route

# We need to add these input/output schemas because the current AgentExecutor
# is lacking in schemas.

class Input(BaseModel):
    input: str
    chat_history: List[BaseMessage] = Field(
        ...,
        extra={"widget": {"type": "chat", "input": "location"}},
    )


class Output(BaseModel):
    output: str


add_routes(
    app,
    agent_executor.with_types(input_type=Input, output_type=Output),
    path="/agent",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

#### Playground
- http://localhost:8000/agent/playground/ 접속
- Input : LangSmith 가 테스트를 어떻게 도와줄 수 있나요?

#### Client

In [46]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/agent/")
remote_chain.invoke(
    {
        "input": "LangSmith 가 테스트를 어떻게 도와줄 수 있나요?",
        "chat_history": [],  # Providing an empty list as this is the first call
    }
)

{'output': 'LangSmith는 다양한 분야에서 도움을 줄 수 있는 인공지능 도구입니다. 예를 들어, LangSmith를 사용하여 정보를 검색하거나 특정 주제에 대한 질문에 답변을 얻을 수 있습니다. 또한 LangSmith를 활용하여 문서 요약, 번역, 언어 감지, 감정 분석 등 다양한 작업을 수행할 수 있습니다. 어떤 도움이 필요한지 알려주시면 더 자세히 안내해 드릴 수 있습니다.'}

## 6. 생성형 AI 활용 실습  
- 1. server.py 코드 설명   
- 2. server.py 코드 리뷰
- 3. server.py 코드 리팩토링
- 4. server.py 주석 추가